# Creamos 1 millón de documentos en colección llamada rendimiento

In [3]:
#Conexión a la bbdd y selección de colección
from pymongo import MongoClient
# ojo con la IP del servidor de base de datos. Comprobar en el servicio docker mongodb
cliente = MongoClient('172.18.0.4', 27017, 
                      username='root', password='abc123', authSource='admin'
                     )

bbdd = cliente.pruebas
coleccion = bbdd.rendimiento

In [4]:
#Este bloque genera 1MM de documentos y los inserta en la bbdd
from random import seed
from random import random
# seed random number generator
seed(1)

for i in range(1000000):
    coleccion.insert_one({'IDCliente': i, 'Número': random()*100})

OperationFailure: Authentication failed., full error: {'ok': 0.0, 'errmsg': 'Authentication failed.', 'code': 18, 'codeName': 'AuthenticationFailed'}

In [ ]:
#Comprobamos los índices actuales
coleccion.index_information()

# Comprobamos tiempos sin índice

In [ ]:
%%time
#Comprobamos tiempo para recuperar un rango de valores
documentos = coleccion.find({ 'Número': { '$gt': 3, '$lt': 3.001 } })
for documento in documentos:
    print(documento)

In [ ]:
%%time
#Medimos el tiempo para obtener el documento con el valor mínimp
documentos = coleccion.find().sort('Número',1).limit(1)
for documento in documentos:
    print(documento)

# Tiempos tras creación de índice por campo Número

In [ ]:
#Comprobamos los índices existentes ahora
coleccion.index_information()

In [ ]:
%%time
#Comprobamos tiempo para recuperar un rango de valores
documentos = coleccion.find({ 'Número': { '$gt': 3, '$lt': 3.001 } })
for documento in documentos:
    print(documento)

In [ ]:
%%time
#Medimos el tiempo para obtener el documento con el valor mínimp
documentos = coleccion.find().sort('Número',1).limit(1)
for documento in documentos:
    print(documento)

# Análisis de planes de ejecución
En el aula lo hacemos utilizando  la consola de Mongo, pero se indican aquí los comandos equivalentes.

In [ ]:
# Eliminamos el índice
coleccion.drop_index('Número_1')

In [ ]:
#Obtenemos el plan de ejecución
coleccion.find({ 'Número': { '$gt': 3, '$lt': 3.001 } }).explain()

In [ ]:
#Volvemos a crear el índice
coleccion.create_index([('Número', 1)])

In [ ]:
#Vemos ahora cómo ha cambiado el plan
coleccion.find({ 'Número': { '$gt': 3, '$lt': 3.001 } }).explain()

# Trabajando con Replica-sets

In [ ]:
from pymongo import MongoClient

#Conexión a la bbdd
cliente = MongoClient('172.18.0.3', 27017, 
                      username='root', password='abc123', authSource='admin',
                      replicaset='miReplica',
                      readPreference='secondaryPreferred'
                     )


In [ ]:
#Comprobamos que ha descubierto automáticamente el resto de nodos
print(cliente.nodes)

In [ ]:
#Hacemos una consulta
bbdd = cliente.tienda
coleccion = bbdd.pedidos
print(coleccion.find_one())

In [ ]:
#Una vez parado el primario, comprobamos los nodos reconocidos
#Comprobamos que ha descubierto automáticamente el resto de nodos
print(cliente.nodes)

In [ ]:
#Repetimos la consulta para verificar que podemos seguir trabajando
print(coleccion.find_one())

In [ ]:
#Y hacemos una inserción para comprobar que hay un nuevo primario
# (si no hubiese primario no podrías modificar)
resultado = coleccion.insert_one({'IDCliente': 34, 'test': 1})
print(resultado.inserted_id)

# Sharding

## Conexión a mongos

In [ ]:
from pymongo import MongoClient

#Utiliza la IP de tu anfitrión
cliente = MongoClient('mongos', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

## Inserción de datos de clientes y de pedidos

In [ ]:
#Inserción de documentos de clientes
documentos_clientes = [
    {'IDCliente': 1, 'Nombre': "Juan", 'Apellidos': "Fernández"},
    {'IDCliente': 2, 'Nombre': "María", 'Apellidos': "Fernández"},
    {'IDCliente': 3, 'Nombre': "Carolina", 'Apellidos': "Pérez"}
]

resultado = coleccion_clientes.insert_many(documentos_clientes)
print("Se han insertado",len(resultado.inserted_ids),"clientes")

In [ ]:
#Inserción de documentos de pedidos
documentos_pedidos = [
    {'IDPedido': 1, 'IDCliente': 1, 'Importe': 3.24, 'Ciudad': "Vigo"},
    {'IDPedido': 2, 'IDCliente': 1, 'Importe': 8.01, 'Ciudad': "Pontevedra"},
    {'IDPedido': 3, 'IDCliente': 3, 'Importe': 28.12, 'Ciudad': "A Coruña"},
    {'IDPedido': 4, 'IDCliente': 1, 'Importe': 56.78, 'Ciudad': "Vigo"},
    {'IDPedido': 5, 'IDCliente': 2, 'Importe': 0.12, 'Ciudad': "Madrid"},
    {'IDPedido': 6, 'IDCliente': 3, 'Importe': 99.45, 'Ciudad': "Barcelona"},
    {'IDPedido': 7, 'IDCliente': 3, 'Importe': 2.1, 'Ciudad': "Valencia"},
    {'IDPedido': 8, 'IDCliente': 1, 'Importe': 9, 'Ciudad': "Ourense"},
    {'IDPedido': 9, 'IDCliente': 1, 'Importe': 32.56, 'Ciudad': "Lugo"},
    {'IDPedido': 10, 'IDCliente': 3, 'Importe': 5.45, 'Ciudad': "Santiago"},
]

resultado = coleccion_pedidos.insert_many(documentos_pedidos)
print("Se han insertado",len(resultado.inserted_ids),"pedidos")

## Comprobación de datos insertados

In [ ]:
#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")

## Conexión a mongoshard1 y cuenta de documentos

In [ ]:
from pymongo import MongoClient

#Como JupyterLab y los nodos de Mongo están conectados a través de la red interna
# establecemos la conexión a travñes de dicha red. Todos escuchan en esa red en el puerto 27017.
# Si te quisieses conectar a través del anfitrión, debes hacerlo al puerto mapeado, en esta caso
# al 27019
cliente = MongoClient('mongoshard1', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")

## Conexión a mongoshard2 y cuenta de documentos

In [ ]:
from pymongo import MongoClient

#Como JupyterLab y los nodos de Mongo están conectados a través de la red interna
# establecemos la conexión a travñes de dicha red. Todos escuchan en esa red en el puerto 27017.
# Si te quisieses conectar a través del anfitrión, debes hacerlo al puerto mapeado, en esta caso
# al 27020
cliente = MongoClient('mongoshard2', 27017)

#Creamos la instancia para interactuar con la colección de clientes y pedidos
bbdd = cliente.tienda
coleccion_clientes = bbdd.clientes
coleccion_pedidos = bbdd.pedidos

#Número de documentos en colección clientes
num_clientes = coleccion_clientes.count_documents({})
print("Hay",num_clientes,"clientes")

#Número de documentos en colección pedidos
num_pedidos = coleccion_pedidos.count_documents({})
print("Hay",num_pedidos,"pedidos")